In [ ]:
import cv2
import numpy as np
import os
import math # For angle calculations
import joblib # For loading models and scalers
import pandas as pd # Used to define feature_columns consistently

# --- Configuration: Model & Scaler Paths ---
# IMPORTANT: These paths must point to where your trained model and scaler are saved.
MODEL_DIR = "model2" # Directory created by train_svr_model.py
SVR_MODEL_FILENAME = "svm_lane_deviation_model.joblib"
SCALER_FILENAME = "scaler_lane_deviation.joblib"

# --- Configuration: Unseen Data Folder ---
# IMPORTANT: Adjust this path to the folder containing your unseen images.
UNSEEN_DATA_FOLDER = "C:/Users/BCI-Lab/Downloads/teamA_dataset/_out_dataset/good_data"

# --- Configuration: Hardcoded Parameters (MUST MATCH THOSE USED FOR FEATURE EXTRACTION & TRAINING) ---
# Copy these values EXACTLY from your 'feature_extractor.py' or 'train_svr_model.py' script.
HARDCODED_PARAMETERS = {
    # HSV Color Space
    "hsv_lower": np.array([6,62, 155]),
    "hsv_upper": np.array([13, 106, 255]),

    # CLAHE Parameters
    "clahe_clip_limit": 93.0,
    "clahe_tile_grid_size": 20,

    # Processing Scale (0.01 to 1.0)
    "processing_scale_percent": 1.0,

    # Morphological Kernel for Color Mask
    "color_morph_kernel_size": 2,

    # Edge Pre-processing Filter
    "use_bilateral_filter": 1,
    "bilateral_d": 14,
    "bilateral_sigma_color": 49,
    "bilateral_sigma_space": 129,

    # Canny Edge Detector Thresholds
    "canny_thresh1": 36,
    "canny_thresh2": 107,

    # Morphological Kernel for Edge Mask
    "edge_morph_kernel_size": 19,

    # Hough Line Transform Parameters
    "hough_threshold": 8,
    "hough_min_length": 32,
    "hough_max_gap": 5,

    # Line Filtering Parameters
    "max_line_angle_deg": 79, # Filter lines by angle (e.g., 20 degrees from horizontal)

    # Image Cropping Parameters (in percentage of original image dimensions)
    "crop_percent_top": 56,
    "crop_percent_left": 15,
    "crop_percent_right": 15,
}


# --- Helper Function to Load Model and Scaler ---
def load_model_and_scaler(model_dir, model_name, scaler_name):
    """Loads the pre-trained SVR model and StandardScaler."""
    try:
        model_path = os.path.join(model_dir, model_name)
        scaler_path = os.path.join(model_dir, scaler_name)
        
        loaded_model = joblib.load(model_path)
        loaded_scaler = joblib.load(scaler_path)
        print(f"✅ Successfully loaded model from: {model_path}")
        print(f"✅ Successfully loaded scaler from: {scaler_path}")
        return loaded_model, loaded_scaler
    except FileNotFoundError as e:
        print(f"ERROR: Model or scaler file not found. Ensure '{model_dir}' exists and contains '{model_name}' and '{scaler_name}'.")
        print(f"Details: {e}")
        exit()
    except Exception as e:
        print(f"ERROR: An error occurred while loading the model or scaler: {e}")
        exit()

# --- Feature Columns (MUST BE IN THE EXACT SAME ORDER AS USED FOR TRAINING) ---
# This list must precisely match the 'feature_columns' in your train_svr_model.py script.
# IMPORTANT: Update these feature columns based on your new extraction logic!
FEATURE_COLUMNS = [
    "lane_centroid_x_cropped_px",
    "num_detected_lines",
    "avg_line_slope_deg",
    "avg_line_length_px",
    "avg_line_x_at_bottom_px",
    "avg_line_x_at_top_px",
    "final_mask_white_pixels",
    "final_mask_centroid_x_px",
    "final_mask_centroid_y_px",
]


# --- Function to Extract Features from a Single Image (Modified for new pipeline) ---
def extract_features_for_prediction(image, params):
    """
    Processes a single image to detect the orange lane line and extract relevant features
    using the new OpenCV pipeline.
    Returns a NumPy array of features and an annotated image.
    """
    
    original_h, original_w, _ = image.shape
    annotated_image = image.copy()

    # Initialize all features with default values
    features_dict = {
        "lane_centroid_x_cropped_px": 0.0,
        "num_detected_lines": 0,
        "avg_line_slope_deg": 0.0,
        "avg_line_length_px": 0.0,
        "avg_line_x_at_bottom_px": 0.0,
        "avg_line_x_at_top_px": 0.0,
        "final_mask_white_pixels": 0,
        "final_mask_centroid_x_px": 0.0,
        "final_mask_centroid_y_px": 0.0,
    }

    # --- 1. Apply Cropping ---
    crop_y_start = int(original_h * params["crop_percent_top"] / 100)
    crop_x_start = int(original_w * params["crop_percent_left"] / 100)
    crop_x_end = original_w - int(original_w * params["crop_percent_right"] / 100)
    if crop_x_start >= crop_x_end: # Fallback for invalid crop
        crop_x_start = 0
        crop_x_end = original_w
    if crop_y_start >= original_h: # Fallback for invalid crop
        crop_y_start = original_h - 1

    cropped_image = image[crop_y_start:original_h, crop_x_start:crop_x_end].copy()
    cropped_h, cropped_w, _ = cropped_image.shape

    # Draw crop zone on annotated image
    cv2.rectangle(annotated_image, (crop_x_start, crop_y_start), (crop_x_end, original_h), (255, 0, 255), 2)
    center_x_cropped_full_image = crop_x_start + (cropped_w // 2)
    cv2.line(annotated_image, (center_x_cropped_full_image, crop_y_start), (center_x_cropped_full_image, original_h), (0, 255, 255), 2)


    if cropped_h == 0 or cropped_w == 0:
        print("Warning: Cropped image has zero height or width. Returning default features.")
        ordered_features_array = np.array([features_dict[col] for col in FEATURE_COLUMNS]).reshape(1, -1)
        return ordered_features_array, annotated_image


    # --- 2. Resize for Processing Performance ---
    processing_width = int(cropped_w * params["processing_scale_percent"])
    processing_height = int(cropped_h * params["processing_scale_percent"])
    if processing_width < 1: processing_width = 1
    if processing_height < 1: processing_height = 1

    cropped_image_for_processing = cv2.resize(cropped_image, (processing_width, processing_height), interpolation=cv2.INTER_LINEAR)

    # --- 3. Illumination Normalization (CLAHE) ---
    hsv_image = cv2.cvtColor(cropped_image_for_processing, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    
    clahe = cv2.createCLAHE(clipLimit=params["clahe_clip_limit"],
                            tileGridSize=(params["clahe_tile_grid_size"], params["clahe_tile_grid_size"]))
    v_clahe = clahe.apply(v)
    
    normalized_hsv = cv2.merge([h, s, v_clahe])
    normalized_image = cv2.cvtColor(normalized_hsv, cv2.COLOR_HSV2BGR)

    # --- 4. Color Masking (HSV on CLAHE-processed image) ---
    hsv_normalized = cv2.cvtColor(normalized_image, cv2.COLOR_BGR2HSV)
    lower_hsv = params["hsv_lower"]
    upper_hsv = params["hsv_upper"]
    color_mask = cv2.inRange(hsv_normalized, lower_hsv, upper_hsv)

    # --- 5. Morphological Operations on the Color Mask ---
    color_morph_kernel = np.ones((params["color_morph_kernel_size"], params["color_morph_kernel_size"]), np.uint8)
    color_mask_morphed = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, color_morph_kernel, iterations=1)
    color_mask_morphed = cv2.dilate(color_mask_morphed, color_morph_kernel, iterations=1)
    color_mask_morphed = cv2.morphologyEx(color_mask_morphed, cv2.MORPH_CLOSE, color_morph_kernel, iterations=1)
    
    # --- 6. Edge Detection (Canny) ---
    gray_normalized = cv2.cvtColor(normalized_image, cv2.COLOR_BGR2GRAY)
    
    if params["use_bilateral_filter"]:
        filtered_gray = cv2.bilateralFilter(gray_normalized, params["bilateral_d"],
                                            params["bilateral_sigma_color"], params["bilateral_sigma_space"])
    else:
        filtered_gray = cv2.GaussianBlur(gray_normalized, (5, 5), 0)

    edge_mask = cv2.Canny(filtered_gray, params["canny_thresh1"], params["canny_thresh2"])
    edge_morph_kernel = np.ones((params["edge_morph_kernel_size"], params["edge_morph_kernel_size"]), np.uint8)
    edge_mask_morphed = cv2.morphologyEx(edge_mask, cv2.MORPH_CLOSE, edge_morph_kernel, iterations=1)

    # --- 7. Combine Masks ---
    final_mask = cv2.bitwise_and(color_mask_morphed, edge_mask_morphed)

    # --- 8. Hough Line Transform ---
    lines = cv2.HoughLinesP(final_mask, 1, np.pi / 180,
                            params["hough_threshold"],
                            minLineLength=params["hough_min_length"],
                            maxLineGap=params["hough_max_gap"])

    # --- Calculate mask centroid if there are white pixels ---
    if np.count_nonzero(final_mask) > 0:
        features_dict["final_mask_white_pixels"] = np.count_nonzero(final_mask)
        # Find coordinates of all white pixels
        coords = cv2.findNonZero(final_mask)
        # Calculate centroid
        mask_cx = int(np.mean(coords[:,:,0]))
        mask_cy = int(np.mean(coords[:,:,1]))
        # Scale mask centroid back to cropped_image dimensions for consistency
        scale_x_mask = cropped_w / processing_width
        scale_y_mask = cropped_h / processing_height
        features_dict["final_mask_centroid_x_px"] = int(mask_cx * scale_x_mask)
        features_dict["final_mask_centroid_y_px"] = int(mask_cy * scale_y_mask)

        # Draw mask centroid on the annotated image (adjusted for full image coordinates)
        mask_cx_full_image = features_dict["final_mask_centroid_x_px"] + crop_x_start
        mask_cy_full_image = features_dict["final_mask_centroid_y_px"] + crop_y_start
        cv2.circle(annotated_image, (mask_cx_full_image, mask_cy_full_image), 8, (255, 0, 0), -1) # Blue circle for mask centroid
        cv2.putText(annotated_image, f"Mask CX: {mask_cx_full_image}", (mask_cx_full_image + 15, mask_cy_full_image - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


    filtered_lines_data = [] # Store (x1, y1, x2, y2, angle_deg, length)
    if lines is not None:
        # Scale coordinates back to original cropped_image dimensions
        scale_x_line = cropped_w / processing_width
        scale_y_line = cropped_h / processing_height
        
        for line in lines:
            x1_scaled, y1_scaled, x2_scaled, y2_scaled = line[0]
            
            x1 = int(x1_scaled * scale_x_line)
            y1 = int(y1_scaled * scale_y_line)
            x2 = int(x2_scaled * scale_x_line)
            y2 = int(y2_scaled * scale_y_line)

            # Ensure no division by zero for angle calculation
            if (x2 - x1) == 0 and (y2 - y1) == 0:
                angle_deg = 0.0
            else:
                angle_rad = math.atan2(y2 - y1, x2 - x1)
                angle_deg = math.degrees(angle_rad)
            
            length = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

            # Original angle filtering logic (keeping lines close to horizontal)
            if (abs(angle_deg) < params["max_line_angle_deg"] or
                abs(angle_deg - 180) < params["max_line_angle_deg"] or
                abs(angle_deg + 180) < params["max_line_angle_deg"]):
                filtered_lines_data.append((x1, y1, x2, y2, angle_deg, length))
                # Draw filtered lines on the annotated image (adjusted for full image coordinates)
                cv2.line(annotated_image, 
                         (x1 + crop_x_start, y1 + crop_y_start), 
                         (x2 + crop_x_start, y2 + crop_y_start), 
                         (0, 255, 0), 2) # Green lines

    if filtered_lines_data:
        features_dict["num_detected_lines"] = len(filtered_lines_data)
        
        all_midpoints_x = []
        all_angles = []
        all_lengths = []
        all_bottom_xs = [] # X-coordinate at y=cropped_h-1
        all_top_xs = []    # X-coordinate at y=0

        for x1, y1, x2, y2, angle_deg, length in filtered_lines_data:
            all_midpoints_x.append((x1 + x2) // 2)
            all_angles.append(angle_deg)
            all_lengths.append(length)

            # Calculate X-coordinate at the bottom (cropped_h-1) and top (0) of the cropped image
            if abs(x2 - x1) > 0.1: # Not a perfectly vertical line (avoid div by zero)
                slope = (y2 - y1) / (x2 - x1)
                if abs(slope) > 1e-6: # Avoid division by zero for horizontal lines
                    # X at bottom (y=cropped_h-1)
                    x_at_bottom = ((cropped_h - 1) - y1) / slope + x1
                    all_bottom_xs.append(x_at_bottom)

                    # X at top (y=0)
                    x_at_top = (0 - y1) / slope + x1
                    all_top_xs.append(x_at_top)
                else: # Nearly horizontal line, use average X
                    avg_x = (x1 + x2) / 2
                    all_bottom_xs.append(avg_x)
                    all_top_xs.append(avg_x)
            else: # Vertical line
                all_bottom_xs.append(x1) # For vertical line, x is constant
                all_top_xs.append(x1)

        features_dict["lane_centroid_x_cropped_px"] = float(np.mean(all_midpoints_x))
        features_dict["avg_line_slope_deg"] = float(np.mean(all_angles))
        features_dict["avg_line_length_px"] = float(np.mean(all_lengths))
        
        if all_bottom_xs:
            features_dict["avg_line_x_at_bottom_px"] = float(np.mean(all_bottom_xs))
        if all_top_xs:
            features_dict["avg_line_x_at_top_px"] = float(np.mean(all_top_xs))

        # Draw centroid of filtered lines on annotated image
        cx_lines_full_image = int(features_dict["lane_centroid_x_cropped_px"]) + crop_x_start
        # Use the middle of the cropped region for Y, or adjust as needed for visual
        cy_lines_full_image = int(cropped_h / 2) + crop_y_start 
        cv2.circle(annotated_image, (cx_lines_full_image, cy_lines_full_image), 6, (0, 0, 255), -1) # Red circle for line centroid
        cv2.putText(annotated_image, f"Line CX: {cx_lines_full_image}", (cx_lines_full_image + 10, cy_lines_full_image + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)


    # Convert features_dict to an ordered NumPy array based on FEATURE_COLUMNS
    ordered_features_array = np.array([features_dict[col] for col in FEATURE_COLUMNS]).reshape(1, -1)
    
    return ordered_features_array, annotated_image # Return features and the image with annotations


# --- Main Execution Block ---
if __name__ == "__main__":
    print("--- Starting Lane Deviation Prediction on Unseen Data ---")

    # Load the trained model and scaler
    svr_model, scaler = load_model_and_scaler(MODEL_DIR, SVR_MODEL_FILENAME, SCALER_FILENAME)

    # Get list of unseen images
    unseen_image_files = [f for f in os.listdir(UNSEEN_DATA_FOLDER) if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))]
    if not unseen_image_files:
        print(f"ERROR: No images found in '{UNSEEN_DATA_FOLDER}'. Please check the path.")
        exit()

    print(f"Found {len(unseen_image_files)} unseen images.")
    print("Press 'q' to quit, 'd' or Right Arrow for next image, 'a' or Left Arrow for previous.")

    current_image_index = 0
    cv2.namedWindow("Prediction Output", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Prediction Output", 1280, 720) # Adjust as needed

    while True:
        filename = unseen_image_files[current_image_index]
        image_path = os.path.join(UNSEEN_DATA_FOLDER, filename)

        original_image = cv2.imread(image_path)
        if original_image is None:
            print(f"❌ ERROR: Cannot read {filename}. Skipping to next.")
            current_image_index = (current_image_index + 1) % len(unseen_image_files)
            continue

        # 1. Extract features and get annotated image
        raw_features_array, annotated_display_image = extract_features_for_prediction(original_image, HARDCODED_PARAMETERS)
        
        # Determine if a line was "detected" based on the number of detected lines
        # This acts as the equivalent of 'is_line_detected' from your previous code
        num_detected_lines_feature = raw_features_array[0, FEATURE_COLUMNS.index('num_detected_lines')]
        is_line_detected_flag = 1 if num_detected_lines_feature > 0 else 0

        predicted_deviation = float('nan') # Default
        display_text = ""

        if is_line_detected_flag == 1:
            # 2. Scale the extracted features
            scaled_features = scaler.transform(raw_features_array)

            # 3. Predict deviation score
            predicted_deviation = svr_model.predict(scaled_features)[0] # .predict returns an array, get the first element
            display_text = f"Predicted Deviation: {predicted_deviation:.4f}"
        else:
            display_text = "NO LINE DETECTED (No Prediction)"
            predicted_deviation = 0.0 # Assign a neutral value if no line, for consistency if logged

        print(f"\nProcessing {filename}: {display_text}")

        # Add prediction text to the image
        cv2.putText(annotated_display_image, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
        
        cv2.imshow("Prediction Output", annotated_display_image)

        # --- Keyboard Controls ---
        key = cv2.waitKey(0) & 0xFF # Wait indefinitely for a key press
        if key == ord("q"): # Press 'q' to quit
            break
        elif key in [ord("d"), 83]: # Press 'd' or Right Arrow to go to the next image
            current_image_index = (current_image_index + 1) % len(unseen_image_files)
        elif key in [ord("a"), 81]: # Press 'a' or Left Arrow to go to the previous image
            current_image_index = (current_image_index - 1 + len(unseen_image_files)) % len(unseen_image_files)

    cv2.destroyAllWindows()
    print("\n--- Prediction session ended. ---")

--- Starting Lane Deviation Prediction on Unseen Data ---
✅ Successfully loaded model from: model2\svm_lane_deviation_model.joblib
✅ Successfully loaded scaler from: model2\scaler_lane_deviation.joblib
Found 1291 unseen images.
Press 'q' to quit, 'd' or Right Arrow for next image, 'a' or Left Arrow for previous.

Processing 00420478.png: NO LINE DETECTED (No Prediction)

Processing 00420523.png: NO LINE DETECTED (No Prediction)

Processing 00420564.png: NO LINE DETECTED (No Prediction)

Processing 00420604.png: NO LINE DETECTED (No Prediction)

Processing 00420644.png: NO LINE DETECTED (No Prediction)

Processing 00420684.png: NO LINE DETECTED (No Prediction)

Processing 00420725.png: NO LINE DETECTED (No Prediction)


c:\Users\BCI-Lab\.conda\envs\openCV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Processing 00420765.png: Predicted Deviation: -0.7199

Processing 00420803.png: NO LINE DETECTED (No Prediction)

Processing 00420841.png: NO LINE DETECTED (No Prediction)

Processing 00420803.png: NO LINE DETECTED (No Prediction)

Processing 00420765.png: Predicted Deviation: -0.7199


c:\Users\BCI-Lab\.conda\envs\openCV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Processing 00420803.png: NO LINE DETECTED (No Prediction)

Processing 00420841.png: NO LINE DETECTED (No Prediction)


...


Processing 00422799.png: Predicted Deviation: 0.2447


c:\Users\BCI-Lab\.conda\envs\openCV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Processing 00422840.png: Predicted Deviation: 0.2380


c:\Users\BCI-Lab\.conda\envs\openCV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Processing 00422881.png: Predicted Deviation: 0.2152

Processing 00422922.png: NO LINE DETECTED (No Prediction)

Processing 00422963.png: NO LINE DETECTED (No Prediction)

Processing 00423003.png: NO LINE DETECTED (No Prediction)

Processing 00423044.png: NO LINE DETECTED (No Prediction)

Processing 00423084.png: NO LINE DETECTED (No Prediction)

Processing 00423122.png: NO LINE DETECTED (No Prediction)

Processing 00423161.png: NO LINE DETECTED (No Prediction)

Processing 00423199.png: NO LINE DETECTED (No Prediction)
